<div style="background-color:#e0f0ff; padding:20px; border-radius:10px;">
  <b style="color:#003366; font-size:2.5em;">Sticky Prices, Shifting Behaviour: Untangling Supply and Demand in U.S. Gasoline Markets</b>
</div>

In this project, we analyse the chicken-and-egg problem of whether supply lags demand or demand lags supply. 

## **The Theory Behind Our Project**

### Using a Keynesian Macroeconomic Lens

From a Keynesian perspective, especially in the **short run**, prices — particularly in **administered markets** like gasoline — tend to be **sticky**. This has several important implications:

- **Sticky prices** imply that **quantities (e.g., consumption)** adjust in response to prices, rather than prices responding quickly to shifts in quantity demanded.
- Thus, we might expect **prices to lead quantities**, not the reverse.

### Implication for Gasoline Markets

If this framework holds:

- **Demand should react to price**, not vice versa.

However, **real-world dynamics** — especially in volatile markets — can complicate this view:

- **Supply shocks** such as:
  - Refinery outages
  - Geopolitical tensions  
  Often lead to **sudden price jumps** independent of demand behavior.

- **Consumer habits and macroeconomic conditions** like:
  - Seasonal driving patterns
  - Recessions
  - Government stimulus checks  
  Can significantly **influence gasoline demand**, potentially leading quantity to drive price in some contexts.

<div style="border-left: 5px solid #2196F3; background-color: #E3F2FD; padding: 15px; margin-bottom: 20px;">
  <h3><strong>Hypothesis</strong></h3>
  <ul>
    <li><strong>H₀ (Null):</strong> Demand (measured through consumption) causes gasoline price fluctuations.</li>
    <li><strong>H₁ (Alternative):</strong> Gasoline price changes cause shifts in demand (i.e., driving behaviour).</li>
  </ul>
</div>

In [2]:
# Imports
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt

## **0. Data Preprocessing and Exploratory Data Analysis (EDA)**

We utilize the following datasets to examine trends in gasoline pricing, supply-demand dynamics, and transportation behavior. For each dataset, we identify specific variables of interest:

| **Dataset**                          | **Selected Variable**                                                                 | **Description**                                                                                                              |
|-------------------------------------|----------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------|
| **Weekly Gasoline Prices**          | `Price`                                                                                | Price reported for that week; tracks nominal gasoline prices over time.                                                     |
| **Weekly Supply Estimates**         | `Weekly U.S. Ending Stocks of Total Gasoline (Thousand Barrels)`                      | Total production of conventional motor gasoline, defined as finished motor gasoline not included in oxygenated or reformulated categories. |
| **Monthly Transportation Statistics** | `Highway Vehicle Miles Traveled - All Systems`                                        | The Federal Highway Administration's estimate of vehicle miles traveled on all roads and streets each month.                |

In [1]:
# Import production and consumer data
prices_weekly = pd.read_csv('./data/weekly_gasoline_prices.csv')
supply_weekly = pd.read_csv('./data/weekly_supply_estimates.csv')
transport = pd.read_csv('./data/monthly_transportation_statistics.csv')

NameError: name 'pd' is not defined

In [ ]:
df = prices_weekly[prices_weekly.duplicated(subset='Date', keep=False)]
df

In [ ]:
prices_weekly = prices_weekly.drop(columns=['Type', 'Unit'])

In [ ]:
data = {
    "prices_weekly": prices_weekly,
    "supply_weekly": supply_weekly,
    "transport": transport
}

#### **0.1 Handling Missing Data**

In [ ]:
for name, df in data.items():
    msno.matrix(df)
    plt.title(f"Missing Data Matrix for {name}")
    plt.show()

##### **Rationale for Handling Missing Data**

We treat missing data differently based on the analytical objective:

**1. Exploratory Analysis: Preserve Raw Data**
- **No imputation** applied.
- Retains natural variability and avoids introducing bias.
- Missingness may itself be informative (e.g., market shocks).
- Suitable for correlation, causality testing, and structural analysis.

**2. Predictive Modeling: Apply Imputation**
- **Imputation required** (e.g., forward fill, interpolation).
- Ensures data continuity for models (e.g., regression, forecasting).
- Applied **only after** exploratory analysis.

| Analysis Type         | Missing Data Approach | Rationale                                  |
|-----------------------|------------------------|--------------------------------------------|
| Exploratory/Structural| Leave as-is            | Preserve signal integrity, avoid bias      |
| Predictive Modeling   | Impute as needed       | Ensure continuity for algorithmic use      |

#### **0.2 Pre-processing**

In [ ]:
# Convert all dates to datetime and handle potential errors
def clean_prices_weekly(df):
    # Remove any strange characters in column names
    df.columns = df.columns.str.strip().str.replace('\\s+', ' ', regex=True)
    
    # Convert date to datetime
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    
    # Clean up Type column if needed
    if 'Type' in df.columns:
        df['Type'] = df['Type'].str.strip()
    
    # Ensure Price column is numeric
    if 'Price' in df.columns:
        df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
    
    return df.dropna(subset=['Date'])  # Remove rows with invalid dates

def clean_supply_weekly(df):
    # Remove any strange characters in column names and simplify names
    df.columns = df.columns.str.strip().str.replace('\\s+', ' ', regex=True)
    
    # Rename columns to be more user-friendly
    rename_dict = {
        'Weekly U.S. Field Production of Crude Oil (Thousand Barrels per Day)': 'Crude_Production',
        'Weekly U.S. Refiner Net Input of Crude Oil (Thousand Barrels per Day)': 'Refiner_Input',
        'Weekly U.S. Imports of Crude Oil (Thousand Barrels per Day)': 'Crude_Imports',
        'Weekly U.S. Exports of Crude Oil (Thousand Barrels per Day)': 'Crude_Exports',
        'Weekly U.S. Imports of Total Gasoline (Thousand Barrels per Day)': 'Gasoline_Imports',
        'Weekly U.S. Refiner and Blender Net Production of Finished Motor Gasoline (Thousand Barrels per Day)': 'Gasoline_Production',
        'Weekly U.S. Percent Utilization of Refinery Operable Capacity (Percent)': 'Refinery_Utilization',
        'Weekly U.S. Ending Stocks of Conventional Motor Gasoline (Thousand Barrels)': 'Gasoline_Stocks'
    }
    
    # Only rename columns that exist in the dataframe
    rename_dict = {k: v for k, v in rename_dict.items() if k in df.columns}
    if rename_dict:
        df = df.rename(columns=rename_dict)
    
    # Convert date to datetime
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    
    # Convert all numeric columns to float
    for col in df.columns:
        if col != 'Date':
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    return df.dropna(subset=['Date'])  # Remove rows with invalid dates

def clean_transport(df):
    # Fix column names
    df.columns = df.columns.str.strip().str.replace('\\s+', ' ', regex=True)
    
    # Convert date to datetime
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    
    # Create MonthStart column for monthly aggregation
    df['MonthStart'] = df['Date'].dt.to_period('M').dt.to_timestamp()
    
    # Rename some columns for clarity
    rename_dict = {
        'Air Safety - General Aviation Fatalities': 'Aviation_Fatalities',
        'Auto sales SAAR (millions)': 'Auto_Sales',
        'Light truck sales SAAR (millions)': 'Light_Truck_Sales',
        'Heavy truck sales SAAR (millions)': 'Heavy_Truck_Sales'
    }
    
    # Only rename columns that exist in the dataframe
    rename_dict = {k: v for k, v in rename_dict.items() if k in df.columns}
    if rename_dict:
        df = df.rename(columns=rename_dict)
    
    # Convert numeric columns to float
    for col in df.columns:
        if col not in ['Date', 'MonthStart', 'Month', 'Year']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    return df.dropna(subset=['Date'])  # Remove rows with invalid dates

In [ ]:
prices_weekly = clean_prices_weekly(prices_weekly)
supply_weekly = clean_supply_weekly(supply_weekly)
transport = clean_transport(transport)

#### **0.3 Merging Datasets**

In [ ]:
# First merge the weekly datasets (prices and supply)
def merge_weekly_data(prices_df, supply_df):
    # Ensure both dataframes are sorted by date
    prices_df = prices_df.sort_values('Date')
    supply_df = supply_df.sort_values('Date')
    
    # Use merge_asof which is ideal for time-series data that may not align perfectly
    combined = pd.merge_asof(
        prices_df,
        supply_df,
        on='Date',
        direction='nearest',
        tolerance=pd.Timedelta('4D')  # Accept matches within 4 days
    )
    
    return combined

# Then add the monthly transport data
def add_transport_data(weekly_df, transport_df):
    # Create a month start column in the weekly dataframe to match with monthly data
    weekly_df['MonthStart'] = weekly_df['Date'].dt.to_period('M').dt.to_timestamp()
    
    # Merge with transport data on MonthStart
    final_df = pd.merge(
        weekly_df,
        transport_df,
        on='MonthStart',
        how='left',
        suffixes=('', '_transport')
    )
    
    # Clean up duplicate columns
    cols_to_drop = [col for col in final_df.columns if col.endswith('_transport') 
                   and col.replace('_transport', '') in final_df.columns]
    final_df = final_df.drop(columns=cols_to_drop)
    
    return final_df

In [ ]:
df = add_transport_data(merge_weekly_data(prices_weekly, supply_weekly), transport)

In [ ]:
 # Drop any rows with all NaN values in the critical columns
critical_cols = ['Price'] + [col for col in df.columns 
                            if 'Production' in col or 'Sales' in col]
df_final = df.copy().dropna(subset=critical_cols, how='all')

### Step 2: Time-Series Analysis

- **Cross-Correlation Function (CCF):**  
  Assess the lead-lag relationship between gasoline prices and demand (product supplied or VMT).

- **Granger Causality Tests:**  
  Determine whether one time series statistically "leads" another — e.g., do past prices predict demand or vice versa?

- **Lagged Regressions:**  
  - Model **demand** as a function of past **gasoline prices**.  
  - Alternatively, model **price** as a function of past **demand**.

### Step 3: Visualizations

- **Dual-Axis Time Plots:**  
  Overlay gasoline prices and demand (or VMT) on the same timeline for direct comparison.

- **Rolling Correlation Windows:**  
  Show how the relationship between price and demand evolves over time.

- **Impulse Response Functions:**  
  If using a Vector Autoregression (VAR) model, trace the impact of shocks in one variable on the other.

## 📊 Visual Narrative

Create a compelling visual story by plotting gasoline price and demand curves during key economic shocks:

- **2008 Financial Crisis**
- **2014 Oil Price Crash**
- **COVID-19 Lockdowns (2020)**
- **2022 Global Energy Crisis**

**Annotations:**  
Highlight significant price spikes and correlate them with observable shifts in consumer behavior (e.g., reduced VMT, panic buying).